In [1]:
import numpy as np

import math

import time

import pandas as pd

import near_optimal

In [2]:
def benchmark_speed(dataset, lsh_kde, L, bandwidth):
    N = dataset.shape[0]
    dim = dataset.shape[1]
    construction_start = time.process_time()
    lsh = lsh_kde(dataset, bandwidth, L)
    construction_end = time.process_time() - construction_start

    n_sample = 10
    test_sample = np.random.choice(N, n_sample, replace=False)
    query_avg = 0.0
    for row_num in test_sample:
        row = dataset[row_num, :]
        query_start = time.process_time()
        lsh.kde(row)
        query_end = time.process_time() - query_start
        query_avg = query_avg + query_end / n_sample
    return construction_end, query_avg


def benchmark_accuracy(dataset, lsh_kde, L, bandwidth, kde):
    N = dataset.shape[0]
    dim = dataset.shape[1]
    
    lsh = lsh_kde(dataset, bandwidth, L)
    n_sample = 10
    test_sample = np.random.choice(N, n_sample, replace=False)
    deviation = 0.0
    for row_num in test_sample:
        row = dataset[row_num, :]
        kde_estimate = lsh.kde(row)
        kde_true = kde(row, dataset, bandwidth)
    
        deviation += np.abs(kde_estimate - kde_true) / kde_true
    
    return deviation / n_sample

## Accuracy Benchmark (change N, D for other experimentation)

In [3]:
acc = []
n = [i for i in range(10, 61)]


for i in n:
    print(i)
    N = 100000
    D = i
    #dataset = np.random.normal(size=(N, D))
    dataset = np.random.uniform(size=(N, D))
    L = 50
    bandwidth = 1
    lsh_kde = near_optimal.FastKDE_NearOptimal
    kde = near_optimal.kde
    acc.append(benchmark_accuracy(dataset, lsh_kde, L, 1, kde))

10


KeyboardInterrupt: 

In [ ]:
write_dat = {
    # change D to other experimentation (such as N)
    "D": n,
    "relative_err": scale_acc,
}

pd_dat = pd.DataFrame(write_dat)

csv_name = "near_opt_D_unif_acc.csv"

pd_dat.to_csv(csv_name, index=False)

## Speed Benchmark (change N, D for other experimentation)

In [4]:
preprocess_time = []
query_time = []
D_s = []

# N:100000 D:50, L 10~100
# N: 10000~1000000 D:50, L:50
# N:100000, D: 10~100, L:50

for i in range(10, 101):
    print(i)
    N = 100000
    D = i
    D_s.append(i)
    #dataset = np.random.normal(size=(N, D))
    dataset = np.random.uniform(size=(N, D))
    L = 50
    bandwidth = 1
    lsh_kde = near_optimal.FastKDE_NearOptimal
    construction, query = benchmark_speed(dataset, lsh_kde, L, bandwidth)
    preprocess_time.append(construction)
    query_time.append(query)

10
11
12


KeyboardInterrupt: 

In [ ]:
write_dat = {
    # change D to other experimentation (such as N)
    "D": D_s,
    "pre": preprocess_time,
    "query": query_time,
}

pd_dat = pd.DataFrame(write_dat)

pd_dat.to_csv("near_opt_D_unif.csv", index=False)